In [1]:
# pip uninstall -y protobuf
# pip uninstall -y tensorflow
# pip uninstall -y tensorflow-gpu
# pip install --upgrade --force-reinstall tensorflow-gpu
# export CUDA_VISIBLE_DEVICES=0

import tflearn
import tensorflow as tf
from tflearn.data_utils import to_categorical, pad_sequences

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
from tflearn.datasets import imdb
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=100000,
                                valid_portion=0.1)
trainX, trainY = train
testX, testY = test
print(test)

trainX = pad_sequences(trainX, maxlen=100, value=0.)
testX = pad_sequences(testX, maxlen=100, value=0.)

trainY = to_categorical(trainY, nb_classes=2)
testY = to_categorical(testY, nb_classes=2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

In [4]:
net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=100000, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16250415715942603552, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6670351073
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13240468572850034665
 physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [6]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, batch_size=128)

Training Step: 1759  | total loss: 0.11476 | time: 31.345s
| Adam | epoch: 010 | loss: 0.11476 - acc: 0.9713 -- iter: 22400/22500
Training Step: 1760  | total loss: 0.11410 | time: 32.520s
| Adam | epoch: 010 | loss: 0.11410 - acc: 0.9718 | val_loss: 0.68424 - val_acc: 0.7936 -- iter: 22500/22500
--


In [7]:
model.predict(testX)

array([[0.9889135 , 0.01108656],
       [0.98889416, 0.01110592],
       [0.00809212, 0.99190795],
       ...,
       [0.00910571, 0.9908943 ],
       [0.05042191, 0.9495781 ],
       [0.01186141, 0.9881386 ]], dtype=float32)